In [ ]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import cv2 as cv
import os
import random
import shutil

In [ ]:
class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (0, 0), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img) 
            
        return img

    def generate_image_dataset(self):
        if not os.path.exists("images"):
            os.mkdir("images")
        while(True):
            img = self.get_screenshot()
            im = Image.fromarray(img[..., [2, 1, 0]])
            im.save(f"./images/img_{len(os.listdir('images'))}.jpeg")
            sleep(1)
    
    def get_window_size(self):
        return (self.w, self.h)

In [ ]:
class LabelUtils:

    def create_shuffled_images_folder(self):
        if not os.path.exists("shuffled_images"):
            os.mkdir("shuffled_images")

        image_files = [f for f in os.listdir("images") if f.endswith(".jpg")]
        random.shuffle(image_files)

        for img in image_files:
            os.rename(f"images/{img}", f"shuffled_images/img_{len(os.listdir('shuffled_images'))}.jpg")

    def create_labeled_images_zip_file(self):
        if not os.path.exists("obj"):
            os.mkdir("obj")

        file_prefixes = [f.split('.')[0] for f in os.listdir("shuffled_images") if f.endswith(".txt")]

        for prefix in file_prefixes:
            os.rename(f"shuffled_images/{prefix}.txt", f"obj/{prefix}.txt")
            os.rename(f"shuffled_images/{prefix}.jpg", f"obj/{prefix}.jpg")
        
        shutil.make_archive('yolov4-tiny/obj', 'zip', 'obj')

In [ ]:
# Execute this cell to generate a dataset of images for the specified window.

window_name = "Diablo II"

wincap = WindowCapture(window_name)
wincap.generate_image_dataset()

In [ ]:
# If you're not going to label all the generated images, make sure to shuffle them.
# Shuffling helps ensure that you will cover a wide range of scenarios. 
# This avoids any bias towards specific patterns or sequences.

# The function below shuffles the images in the images folder and inserts them into the shuffled_images folder.

lbUtils = LabelUtils()
lbUtils.create_shuffled_images_folder()

In [ ]:
# After creating the dataset (and shuffling the images), the next step is to label the images. 
# (I recommend using https://www.makesense.ai/ as demonstrated in the video). 

In [ ]:
# Then, copy the .txt yolo label files into the "shuffled_images" folder and execute the following command
# to generate a zip file with the images and labels inside the yolov4-tiny folder.

lbUtils.create_labeled_images_zip_file()

In [ ]:
# Now, add the "yolov4-tiny" folder to the root directory of your Google Drive
# and proceed to train the model using the "yolo_model_training" 
# notebook in Google Colab.

# Ensure that the "yolov4-tiny" folder has the following structure and files:

# yolov4-tiny/
# ├── obj.data
# ├── obj.names
# ├── obj.zip
# ├── process.py
# ├── yolov4-tiny-custom.cfg
# └── training/
#     └── placeholder.txt

In [ ]:
# After finishing training the model, copy the `.weights` file from the `yolov4-tiny/training` folder
# and the `.cfg` file from the `yolov4-tiny` folder to the same directory as this notebook.

In [3]:
!zip -r obj.zip obj/

'zip' is not recognized as an internal or external command,
operable program or batch file.


'D:\\CHROME DOWNLOADS\\yolo-opencv-detector\\yolov4-tiny\\obj.zip'